In [540]:
#### Preamble ####
# Purpose: Cleans, engineers, and saves the cleaned data 
# Author: Jiazhou(Justin) Bi
# Date: 15 Nov 2024
# Contact: justin.bi@mail.utoronto.ca
# License: None
# Pre-requisites: see requirements.txt
# Any other information needed? None

# Loading the data

In [541]:
import pandas as pd

In [542]:
#Loading the datasets as DataFrame
df_1m = pd.read_parquet('../data/01-raw_data/raw_data_1m.parquet')
df_1h = pd.read_parquet('../data/01-raw_data/raw_data_1h.parquet')
df_1d = pd.read_parquet('../data/01-raw_data/raw_data_1d.parquet')
# print(df_1m.head())
# print(df_1h.head())
# print(df_1d.head())

# Checking for Missing Values

In [543]:
df_1m.isna().sum()

timestamp    0
open         0
high         0
low          0
close        0
volume       0
dtype: int64

In [544]:
df_1h.isna().sum()

timestamp    0
open         0
high         0
low          0
close        0
volume       0
dtype: int64

In [545]:
df_1d.isna().sum()

timestamp    0
open         0
high         0
low          0
close        0
volume       0
dtype: int64

There are no missing values found in these datasets.

# Data Types

This section examines the datatypes of each column and ensures they are appropriate for the analysis.

In [546]:
df_1m.dtypes

timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume              float64
dtype: object

In [547]:
df_1h.dtypes

timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume              float64
dtype: object

In [548]:
df_1d.dtypes

timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume              float64
dtype: object

# Data Validation

This section is to validate if the datasets are consecutive. That is, for the 1-minute timestamp dataset, no minutes should be skipped. Same logic appleis to the 1-hour and 1-day timestamp datasets.

## Validating if any minute is missing

In [549]:
# creating a full range of minutes for the 1-minute dataset
full_minute_range = pd.date_range(start=df_1m['timestamp'].min(), end=df_1m['timestamp'].max(), freq='1min')
# print(full_minute_range)

In [550]:
# Reindexing the DataFrame
df_1m.set_index('timestamp', inplace=True)
df_1m = df_1m.reindex(full_minute_range)
df_1m.index.name = 'index'
df_1m['index'] = df_1m.index
df_1m.reset_index(drop=True, inplace=True)

In [551]:
# Check for missing values across the entire DataFrame
missing_rows = df_1m[df_1m.isna().any(axis=1)]

if missing_rows.empty:
    print("No missing values.")
else:
    print("Rows with missing values:")
    print(missing_rows)

Rows with missing values:
         open  high  low  close  volume               index
29521     NaN   NaN  NaN    NaN     NaN 2017-09-06 16:01:00
29522     NaN   NaN  NaN    NaN     NaN 2017-09-06 16:02:00
29523     NaN   NaN  NaN    NaN     NaN 2017-09-06 16:03:00
29524     NaN   NaN  NaN    NaN     NaN 2017-09-06 16:04:00
29525     NaN   NaN  NaN    NaN     NaN 2017-09-06 16:05:00
...       ...   ...  ...    ...     ...                 ...
2945395   NaN   NaN  NaN    NaN     NaN 2023-03-24 13:55:00
2945396   NaN   NaN  NaN    NaN     NaN 2023-03-24 13:56:00
2945397   NaN   NaN  NaN    NaN     NaN 2023-03-24 13:57:00
2945398   NaN   NaN  NaN    NaN     NaN 2023-03-24 13:58:00
2945399   NaN   NaN  NaN    NaN     NaN 2023-03-24 13:59:00

[8632 rows x 6 columns]


There are missing minutes. We will first mark the missing rows down in a new column called 'was_miss'. 1 indicates the row was empty and 0 means not. Then the missing values are filled with linear interpolation, except the column "volume", which is replaced with 0.

In [552]:
# creating an indicator to flag the missing rows
df_1m['was_missing'] = df_1m.isna().any(axis=1).astype(int)

# Interpolate missing values
df_1m.interpolate(method='linear', inplace=True)
df_1m.loc[df_1m['was_missing'] == 1, 'volume'] = 0

In [553]:
# Check agian for missing values across the entire DataFrame
missing_rows = df_1m[df_1m.isna().any(axis=1)]

if missing_rows.empty:
    print("No missing values.")
else:
    print("Rows with missing values:")
    print(missing_rows)

No missing values.


## Validating if any hour is missing from the 1-hour dataset

In [554]:
# using the same logic as above
full_hour_range = pd.date_range(start=df_1h['timestamp'].min(), end=df_1h['timestamp'].max(), freq='1h')

# Reindexing the DataFrame
df_1h.set_index('timestamp', inplace=True)
df_1h = df_1h.reindex(full_hour_range)
df_1h.index.name = 'index'
df_1h['index'] = df_1h.index
df_1h.reset_index(drop=True, inplace=True)

# Check for missing values across the entire DataFrame
missing_rows_1h = df_1h[df_1h.isna().any(axis=1)]

if missing_rows_1h.empty:
    print("No missing values.")
else:
    print("Rows with missing values:")
    print(missing_rows_1h)

Rows with missing values:
       open  high  low  close  volume               index
493     NaN   NaN  NaN    NaN     NaN 2017-09-06 17:00:00
494     NaN   NaN  NaN    NaN     NaN 2017-09-06 18:00:00
495     NaN   NaN  NaN    NaN     NaN 2017-09-06 19:00:00
496     NaN   NaN  NaN    NaN     NaN 2017-09-06 20:00:00
497     NaN   NaN  NaN    NaN     NaN 2017-09-06 21:00:00
...     ...   ...  ...    ...     ...                 ...
34968   NaN   NaN  NaN    NaN     NaN 2021-08-13 04:00:00
34969   NaN   NaN  NaN    NaN     NaN 2021-08-13 05:00:00
36099   NaN   NaN  NaN    NaN     NaN 2021-09-29 07:00:00
36100   NaN   NaN  NaN    NaN     NaN 2021-09-29 08:00:00
49089   NaN   NaN  NaN    NaN     NaN 2023-03-24 13:00:00

[128 rows x 6 columns]


In [555]:
# applying the same logic as above
df_1h['was_missing'] = df_1h.isna().any(axis=1).astype(int)

# Interpolate missing values
df_1h.interpolate(method='linear', inplace=True)
df_1h.loc[df_1h['was_missing'] == 1, 'volume'] = 0

# Check agian for missing values across the entire DataFrame
missing_rows_1h = df_1h[df_1h.isna().any(axis=1)]

if missing_rows_1h.empty:
    print("No missing values.")
else:
    print("Rows with missing values:")
    print(missing_rows_1h)

No missing values.


## Validating if any day is missing from the 1-day dataset

In [556]:
df_1d.head()

,timestamp,open,high,low,close,volume
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060


In [557]:
# using the same logic as above
full_day_range = pd.date_range(start=df_1d['timestamp'].min(), end=df_1d['timestamp'].max(), freq='1d')

# Reindexing the DataFrame
df_1d.set_index('timestamp', inplace=True)
df_1d = df_1d.reindex(full_day_range)
df_1d.index.name = 'index'
df_1d['index'] = df_1d.index
df_1d.reset_index(drop=True, inplace=True)

# Check for missing values across the entire DataFrame
missing_rows_1d = df_1d[df_1d.isna().any(axis=1)]

if missing_rows_1d.empty:
    print("No missing values.")
else:
    print("Rows with missing values:")
    print(missing_rows_1d)

No missing values.


For the 1-day dataset, no days were missing.

# Adding A Column For Price Change Direction

This subsection creates two new column for each dataset called direction. For direction_t-1, if the closing price is higher than the previous closing price, it is considered that the price has gone up and thus marked as 1 for appreciation. If the closing price is lower than the previous closing price, it is considered that the price has gone down and hence is marked as -1 for depreciation. If the price remains the same,  it is marked as 0 for no movement. Same for direction_t+1, but this indicates that if the price will go up or down in the next tick(timestamp).

In [558]:
# Calculate direction_t-1 and direction_t+1
df_1m['direction_t-1'] = df_1m['close'].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df_1m['direction_t+1'] = df_1m['close'].shift(-1).sub(df_1m['close']).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

# Drop the first and last rows where direction_t-1 or direction_t+1 is NaN
df_1m.dropna(subset=['direction_t-1', 'direction_t+1'], inplace=True)

# Ensure both columns are integers
df_1m['direction_t-1'] = df_1m['direction_t-1'].astype(int)
df_1m['direction_t+1'] = df_1m['direction_t+1'].astype(int)

# Display the DataFrame for verification
df_1m.head(20)

,open,high,low,close,volume,index,was_missing,direction_t-1,direction_t+1
0,4261.48,4261.48,4261.48,4261.48,1.775183,2017-08-17 04:00:00,0,0,0
1,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:01:00,0,0,1
2,4280.56,4280.56,4280.56,4280.56,0.261074,2017-08-17 04:02:00,0,1,-1
3,4261.48,4261.48,4261.48,4261.48,0.012008,2017-08-17 04:03:00,0,-1,0
4,4261.48,4261.48,4261.48,4261.48,0.140796,2017-08-17 04:04:00,0,0,0
5,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:05:00,0,0,0
6,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:06:00,0,0,0
7,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:07:00,0,0,0
8,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:08:00,0,0,0
9,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:09:00,0,0,0


In [559]:
# applying the same logic to the 1-hour and 1-day datasets
df_1h['direction_t-1'] = df_1h['close'].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df_1h['direction_t+1'] = df_1h['close'].shift(-1).sub(df_1h['close']).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df_1h.dropna(subset=['direction_t-1', 'direction_t+1'], inplace=True)
df_1h['direction_t-1'] = df_1h['direction_t-1'].astype(int)
df_1h['direction_t+1'] = df_1h['direction_t+1'].astype(int)

df_1d['direction_t-1'] = df_1d['close'].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df_1d['direction_t+1'] = df_1d['close'].shift(-1).sub(df_1d['close']).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df_1d.dropna(subset=['direction_t-1', 'direction_t+1'], inplace=True)
df_1d['direction_t-1'] = df_1d['direction_t-1'].astype(int)
df_1d['direction_t+1'] = df_1d['direction_t+1'].astype(int)

# Saving the DataFrame as a parquet file

In [560]:
df_1m.to_parquet('../data/02-analysis_data/cleaned_data_1m.parquet', index=False)
df_1h.to_parquet('../data/02-analysis_data/cleaned_data_1h.parquet', index=False)
df_1d.to_parquet('../data/02-analysis_data/cleaned_data_1d.parquet', index=False)